In [ ]:
import numpy as np
import scipy as sp
import scipy.integrate as integrate
import matplotlib.pyplot as plt
%matplotlib
def pert(x):#Eingabe Störpotential
    return eval("lambda x:" + input("Please input a function in numpy notation!"))
x=0
pertfunc=pert(x)

λ = 0.1#Störparameter
L = 1#Länge des Kastens
n = 2#Quantenzahl der ungest. WF
maxk=50#bis welche Quantenzahl entwickelt werden soll
hbar=1#hquer wird 1 gesetzt
m=1#Masse
coeff = np.empty([maxk,maxk])
def psi_0(n,x,L):#ungestörte WF des Teilchen im unendlichen Potkasten
    return np.sqrt(2/L)*np.sin(n*np.pi*x/L)
def E_0(n,hbar,L,m):#ungestörter Energie-EW
    return (hbar*np.pi/L)**2 *(1/(2*m))*(n**2)




it = np.nditer(coeff, flags=['multi_index'], op_flags=['readwrite'])#berechnen der Matrixeinträge des Störoperators in Psi_0-Basis
while not it.finished:
    if it.multi_index[0] == it.multi_index[1]:#Hauptdiagonale muss 0 sein
        it[0]= 0
        it.iternext()
    else:
        it[0] = integrate.quad(lambda x: pertfunc(x)*psi_0(it.multi_index[0]+1,x,L)*psi_0(it.multi_index[1]+1,x,L),0,L)[0]
        it.iternext()

        
def psi_1(n,x,L,hbar,maxk,λ,coeff):#Berechnung der Ersten Störordung 
    a = 0
    for i in range(maxk):
        if i+1 == n:
            continue
        else:
            a +=λ*psi_0(i+1,x,L)*coeff[n-1,i]/(E_0(n,hbar,L,m)-E_0(i+1,hbar,L,m))
    return a
def psi_2(n,x,L,hbar,maxk,λ,coeff):#Berechnung der zweiten Störordnung
    b=0
    for i in range (maxk):
        if i+1 ==n:
            continue
        else:
            for j in range(maxk):
                if j+1 == n:
                    continue
                else:
                    b +=((λ**2)/(E_0(n,hbar,L,m)-E_0(i+1,hbar,L,m)))*(psi_0(i+1,x,L)*coeff[i,j]*coeff[j,n-1]/(E_0(n,hbar,L,m)-E_0(j+1,hbar,L,m))-(coeff[i,n-1]/(E_0(n,hbar,L,m)-E_0(i+1,hbar,L,m)))*(coeff[n-1,n-1]*psi_0(i+1,x,L)+0.5*psi_0(n,x,L)*coeff[n-1,i])) 
    return b

norm = integrate.quad(lambda x: (psi_0(n,x,L)+psi_1(n,x,L,hbar,maxk,λ,coeff)+psi_2(n,x,L,hbar,maxk,λ,coeff))**2,0,L)[0]#Normierung
#Plotten der gestörten und ungestörten WF:
x_data = np.linspace(0,L, num=200)
y_data = (psi_0(n,x_data,L)+psi_1(n,x_data,L,hbar,maxk,λ,coeff)+psi_2(n,x_data,L,hbar,maxk,λ,coeff))/np.sqrt(norm)
psi_0_data = psi_0(n,x_data,L)
plt.figure(figsize=(20, 4))
plt.plot(x_data, y_data**2)
plt.plot(x_data, psi_0_data**2)
plt.show()